In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import det, inv
from sklearn.metrics import roc_curve, auc

P0, P1 = 0.6, 0.4                  # priors
w01 = w02 = w11 = w12 = 0.5        # mixture weights
m01 = np.array([-0.9, -1.1])
m02 = np.array([ 0.8,  0.75])
m11 = np.array([-1.1,  0.9])
m12 = np.array([ 0.9, -0.75])
C = np.diag([0.75, 1.25])          # same covariance for all components
Cinvg = inv(C)
norm_const = 1.0 / (2*np.pi*np.sqrt(det(C)))
rng = np.random.default_rng(7)     # reproducibility

# ------------------------------------
# Gaussian pdf and class pdf functions
# ------------------------------------
def gaussian_pdf(x, mu):
    # x: (..., 2), mu: (2,)
    d = x - mu
    expo = -0.5 * np.einsum('...i,ij,...j', d, Cinvg, d)
    return norm_const * np.exp(expo)

def p_x_given_0(x):
    # mixture for L=0: 0.5*N(m01,C) + 0.5*N(m02,C)
    return 0.5*gaussian_pdf(x, m01) + 0.5*gaussian_pdf(x, m02)

def p_x_given_1(x):
    # mixture for L=1: 0.5*N(m11,C) + 0.5*N(m12,C)
    return 0.5*gaussian_pdf(x, m11) + 0.5*gaussian_pdf(x, m12)

# ----------------------------------------
# Generate validation set D10K_validate
# ----------------------------------------
def sample_class(label, n):
    # pick component with prob 0.5 each
    comp = rng.integers(0, 2, size=n)
    mus = {
        0: [m01, m02],
        1: [m11, m12]
    }
    X = np.empty((n, 2))
    for i in range(n):
        X[i] = rng.multivariate_normal(mus[label][comp[i]], C, method='eigh')
    y = np.full(n, label, dtype=int)
    return X, y

def sample_validation(n=10000):
    n1 = rng.binomial(n, P1)   # draw labels according to priors
    n0 = n - n1
    X0, y0 = sample_class(0, n0)
    X1, y1 = sample_class(1, n1)
    X = np.vstack([X0, X1])
    y = np.concatenate([y0, y1])
    # shuffle
    idx = rng.permutation(n)
    return X[idx], y[idx]

Xv, yv = sample_validation(10000)

# ----------------------------------------
# Bayes score and decisions
# ----------------------------------------
def bayes_score(x):
    # log p(x|1)P1 - log p(x|0)P0  (posterior log-odds up to a constant)
    px1 = p_x_given_1(x)
    px0 = p_x_given_0(x)
    # Numerical safety
    eps = 1e-300
    s = np.log(px1 * P1 + eps) - np.log(px0 * P0 + eps)
    return s

scores = bayes_score(Xv)

# ----------------------------------------
# ROC curve & min-error point
# ----------------------------------------
fpr, tpr, thresholds = roc_curve(yv, scores)  # positive class = 1 by default
roc_auc = auc(fpr, tpr)

# For each threshold, compute empirical 0-1 error on validation
# Decision rule: predict 1 if score >= thr
preds_all = (scores[:, None] >= thresholds[None, :]).astype(int)
# Error rates for each threshold
errors = (preds_all != yv[:, None]).mean(axis=0)
min_idx = np.argmin(errors)
min_err = float(errors[min_idx])
min_fpr, min_tpr = float(fpr[min_idx]), float(tpr[min_idx])
min_thr = float(thresholds[min_idx])

print(f"Estimated min P(error) on D10K validate: {min_err:.4f}")
print(f"Threshold at min-error: {min_thr:.5f}")
print(f"FPR={min_fpr:.4f}, TPR={min_tpr:.4f}: ")
print(f"ROC Area: {roc_auc:.4f}")

# ----------------------------------------
# Plot ROC with min-error point highlighted
# ----------------------------------------
plt.figure(figsize=(5.4, 5))
plt.plot(fpr, tpr, color='darkblue', lw=2, label=f"ROC Area ={roc_auc:.3f}")
plt.plot([0,1],[0,1],'--',lw=1, color='gray', label="chance")

# Red star for min-error point
plt.scatter([min_fpr], [min_tpr], s=90, color='red', marker='*', label="min-P(error) point")

# Text label with numeric min-P(error)
plt.text(min_fpr + 0.03, min_tpr - 0.03,
         f"Pₑ={min_err:.3f}", color='red', fontsize=10, weight='bold')

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Bayes Classifier ROC")
plt.legend(loc='lower right')
plt.grid(True, ls=':', alpha=0.6)
plt.tight_layout()
plt.show()

# ----------------------------------------
# (Optional) Decision boundary overlaid on validation set
# ----------------------------------------
do_boundary_plot = True
if do_boundary_plot:
    from matplotlib.lines import Line2D

    pad = 1.0
    x1min, x1max = Xv[:,0].min()-pad, Xv[:,0].max()+pad
    x2min, x2max = Xv[:,1].min()-pad, Xv[:,1].max()+pad
    xx1, xx2 = np.meshgrid(
        np.linspace(x1min, x1max, 400),
        np.linspace(x2min, x2max, 400),
    )
    grid = np.c_[xx1.ravel(), xx2.ravel()]
    sgrid = bayes_score(grid)
    Z = (sgrid >= 0).astype(int).reshape(xx1.shape)

    plt.figure(figsize=(6.4, 5.4))
    plt.contourf(xx1, xx2, Z, levels=[-0.5, 0.5, 1.5], alpha=0.15)

    # Plot the decision boundary (score == 0). We won't touch cs.collections.
    cs = plt.contour(xx1, xx2, sgrid.reshape(xx1.shape), levels=[0.0], linewidths=2)

    # Scatter validation data
    s0 = plt.scatter(Xv[yv==0,0], Xv[yv==0,1], s=8, alpha=0.6, label="Class 0")
    s1 = plt.scatter(Xv[yv==1,0], Xv[yv==1,1], s=8, alpha=0.6, label="Class 1")

    # Build legend handles/labels and add a proxy line for the boundary
    handles, labels = plt.gca().get_legend_handles_labels()
    boundary_proxy = Line2D([], [], linestyle='-', linewidth=2, label="Bayes boundary (score=0)")
    handles.append(boundary_proxy)
    labels.append("Bayes boundary (score=0)")
    plt.legend(handles, labels)

    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.title("Validation set with Bayes decision boundary")
    plt.grid(True, ls=':', alpha=0.6)
    plt.tight_layout()
    plt.show()
